In [ ]:
! pip install ultralytics wandb roboflow --upgrade --quiet --progress-bar off

  Preparing metadata (setup.py) ... done


* __Dataset Link__: [Roboflow Universe](https://universe.roboflow.com/jizo/aadhar-card-entity-detection)

In [ ]:
import yaml
from roboflow import Roboflow
from ultralytics import YOLO
import wandb

In [ ]:
with open('api-keys.yml') as fp:
    secrets = yaml.load(fp, yaml.SafeLoader)

wandb.login(key=secrets['WANDB'])

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
# download datasets
rf = Roboflow(api_key=secrets['ROBOFLOW'])
project = rf.workspace("jizo").project("aadhar-card-entity-detection")
dataset = project.version(1).download("yolov8", location='datasets')

# get labels and label_ids
labels = sorted(project.classes.keys())
id2label = {0: 'AADHAR_NUMBER', 1: 'DATE_OF_BIRTH', 2: 'GENDER', 3:'NAME', 4:'ADDRESS'}

# generate data.yaml file
with open('data.yaml', 'w') as fp:
    data_yaml = dict(
        path = ".",
        train = "train",
        val = "valid",
        test = "test",
        nc = len(labels),
        names = id2label
    )
    yaml.dump(data_yaml, fp)

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.134 is required but found version=8.0.198, to fix: `pip install ultralytics==8.0.134`



Extracting Dataset Version Zip to datasets in yolov8:: 100%|██████████| 5304/5304 [00:00<00:00, 6680.30it/s]


In [ ]:
run = wandb.init(
    # wandb server data
    job_type = 'train/detect',
    project = "Aadhar-Card",
    name = "fine-tune-v1",

    # training metadata & hyper-parameters
    config = dict(
        metadata = dict(
            model_variant = "YOLOv8-Nano-pretrained",
            dataset_source = "Roboflow Universe",
            dataset_url = "https://universe.roboflow.com/jizo/aadhar-card-entity-detection/dataset/1"
        ),
        model_name = "yolov8n.pt",
        data_yaml = data_yaml,
        hyper_parameters = dict(
            batch = 16,
            epochs = 100,
            optimizer = "AdamW",
            imgsz = 640,
            save_period = 25,
            seed = 42
        )
    )
)

In [ ]:
from ultralytics import YOLO

model = YOLO(run.config['model_name'])

wandb.watch(model)

[]

In [ ]:
model.train(device=0, data = 'data.yaml', project=run.project, name=run.name, **run.config['hyper_parameters'])

In [ ]:
run.finish()